### Installing Packages

#### Conda environnment
You can create the same conda environment that was used to run this notebook using the following command:

```
conda env create -f requirements.yml
```

In [33]:
import pandas as pd
import ast
import numpy as np
import sys
import geopandas
from geopy.distance import geodesic
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.ops import nearest_points
from shapely import wkt

### Method 1: Create GeoPandas objects from scratch
#### Import data

In [13]:
properati = pd.read_csv('properati_data/sell/combined_sell.csv', encoding= "ISO-8859-1")
prop_data = pd.read_csv('prop_data/cleaned_data.csv', encoding= "ISO-8859-1")
census = pd.read_csv('census/cleaned_data2.csv',encoding= "ISO-8859-1")

### Create GeoPandas dataframes

In [10]:
def modify_poly(data):
    coord_list = []
    for coord in data:
        coord_list.append(tuple(coord))
    return coord_list

In [14]:
# create a new column called coordinates on properati data frame
properati['coordinates'] = list(zip(properati.lon, properati.lat))
properati['coordinates'] = properati['coordinates'].apply(Point)

# convert properati data into geopandas dataframe
properati_gdf = geopandas.GeoDataFrame(properati, geometry='coordinates')

# convert the longitudes and latitudes in the properaty values dataframe into float (they were originally in string)
prop_data.Latitude = pd.to_numeric(prop_data.Latitude, errors='coerce')
prop_data.Longitude = pd.to_numeric(prop_data.Longitude, errors='coerce')

print ('Number of rows in Properati dataset before merge: {}'.format(len(properati_gdf)))

# convert property values data into geopandas dataframe
prop_data['coordinates'] = list(zip(prop_data.Longitude, prop_data.Latitude))
#prop_data['coordinates'] = list(zip(prop_data.Latitude, prop_data.Longitude))
prop_data['coordinates'] = prop_data['coordinates'].apply(Point)
prop_data_gdf = geopandas.GeoDataFrame(prop_data, geometry='coordinates')


# convert census polygon data into tuples and polygon
census.coordinates = census.coordinates.apply(lambda x: ast.literal_eval(x)[0])
census.coordinates = census.coordinates.apply(lambda x: modify_poly(x))
census.coordinates = census.coordinates.apply(Polygon)

census_gdf = geopandas.GeoDataFrame(census, geometry='coordinates')
print ('Number of rows in census dataset: {}'.format(len(census_gdf)))

Number of rows in Properati dataset before merge: 1871732
Number of rows in census dataset: 3552


### Filter only data that is in the city of the Buenos Aires

This is done by doing a spatial join between the Properati dataset and the census dataset.

**Note**: This takes a while.

In [15]:
properati_census_gdf = geopandas.sjoin(properati_gdf, census_gdf, how="inner", op='within')
print ('{:.2f}%'.format(len(properati_census_gdf)/len(properati_gdf)))

print ('Properati dataset after merge: {}'.format(len(properati_census_gdf)))

/anaconda3/envs/geopandas/lib/python3.7/site-packages/numpy/lib/function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


0.25%
Properati dataset after merge: 464869


In [17]:
# Export the merged data to a file. 
temp_df = pd.DataFrame(properati_census_gdf)
temp_df.to_csv('properati_data/sell/sell_census.csv')

### Method 2: Import already merged data



#### Import Properati dataset merged with census data

This is a dataset that includes Properati data within the Autonomous City of Buenos Aires.

In [34]:
temp_df = pd.read_csv('properati_data/sell/sell_census.csv')
temp_df = temp_df.drop_duplicates(subset='id_left')
temp_df.coordinates = list(zip(temp_df.lon, temp_df.lat))
temp_df.coordinates = temp_df.coordinates.apply(Point)
properati_census_gdf = geopandas.GeoDataFrame(temp_df, geometry='coordinates')
print (len(properati_census_gdf))



116425


In [20]:
properati_census_gdf.head(5)

,Unnamed: 0,dataset_date,id_left,created_on,operation,property_type,place_name,country_name,state_name,geonames_id,...,PROV,RADIO,RADPAIS_,REDCODE,Regular Percent,Regular Quantile,TIPO,Uninhabited Percent,Uninhabited Quantile,type
0,1,201501,155a59477c557a77de931cb2ef91d6a9b23019d7,8/8/2014,sell,apartment,Barracas,Argentina,Capital Federal,3436134.0,...,2.0,3.0,34863.0,20041303.0,81.3,1,NaN,1.3,1,Polygon
1,13547,201501,cf5823f73347ea539a10738c219624a1b9a85737,8/8/2014,sell,apartment,Barracas,Argentina,Capital Federal,3436134.0,...,2.0,3.0,34863.0,20041303.0,81.3,1,NaN,1.3,1,Polygon
2,38029,201502,a5f052a6dea369a73d2f85326d88944659d44d86,11/27/2015,sell,house,Barracas,Argentina,Capital Federal,3436134.0,...,2.0,3.0,34863.0,20041303.0,81.3,1,NaN,1.3,1,Polygon
3,51933,201502,03bb5c5811d5db8617aaf9ca8b4ce93acfa2b1eb,11/28/2015,sell,store,Barracas,Argentina,Capital Federal,3436134.0,...,2.0,3.0,34863.0,20041303.0,81.3,1,NaN,1.3,1,Polygon
4,109853,201502,5e5c722f19dc028609bcdcae50bc478b3d3c9a41,11/10/2015,sell,apartment,Barracas,Argentina,Capital Federal,3436134.0,...,2.0,3.0,34863.0,20041303.0,81.3,1,NaN,1.3,1,Polygon


#### import barrios geojson

In [35]:
barrios = pd.read_csv('shape files/barrios.csv')
barrios.WKT = barrios.WKT.apply(wkt.loads)

# normalize barrio names
barrios['barrio'] = (barrios['barrio'].apply(lambda x: str(x.replace('VELEZ SARSFIELD', 'VELEZ SARFIELD'))))
barrios['barrio'] = (barrios['barrio'].apply(lambda x: str(x.replace('VILLA GRAL. MITRE', 'VILLA GENERAL MITRE'))))
barrios['barrio'] = (barrios['barrio'].apply(lambda x: str(x.replace('VILLA GRAL. MITR', 'VILLA GENERAL MITRE'))))
barrios['barrio'] = (barrios['barrio'].apply(lambda x: str(x.replace('VILLA GRAL MITRE', 'VILLA GENERAL MITRE'))))
barrios['barrio'] = (barrios['barrio'].apply(lambda x: str(x.replace('VERSALLES', 'VERSAILLES'))))
barrios['barrio'] = (barrios['barrio'].apply(lambda x: str(x.replace('MONSERRAT', 'MONTSERRAT'))))

barrio_df = pd.read_csv('barrio_table.csv')
barrio_df['b_id'] = [int(barrio_df[barrio_df.Barrio == i].id) for i in barrio_df.Barrio]
barrio_df['barrio'] = barrio_df.Barrio

barrios = pd.merge(barrios, barrio_df, on='barrio', how='left')
df = barrios[barrios.id.isnull()]

barrios_gdf = geopandas.GeoDataFrame(barrios, geometry='WKT')

In [36]:
barrio_df.head(5)

,Barrio,id,b_id,barrio
0,PALERMO,0,0,PALERMO
1,BELGRANO,1,1,BELGRANO
2,NUÑEZ,2,2,NUÑEZ
3,VILLA ORTUZAR,3,3,VILLA ORTUZAR
4,BALVANERA,4,4,BALVANERA


In [37]:
properati_census_gdf = properati_census_gdf.drop(columns=['id_right', 'index_right'], axis=1)
final_gdf = geopandas.sjoin(properati_census_gdf , barrios_gdf, how="inner", op='within')


In [24]:
final_gdf.head(5)

,Unnamed: 0,dataset_date,id_left,created_on,operation,property_type,place_name,country_name,state_name,geonames_id,...,Uninhabited Quantile,type,index_right,barrio,comuna,perimetro,area,Barrio,id,b_id
0,1,201501,155a59477c557a77de931cb2ef91d6a9b23019d7,8/8/2014,sell,apartment,Barracas,Argentina,Capital Federal,3436134.0,...,1,Polygon,29,BARRACAS,4,13018.210271,7.961000e+06,BARRACAS,32,32
1,13547,201501,cf5823f73347ea539a10738c219624a1b9a85737,8/8/2014,sell,apartment,Barracas,Argentina,Capital Federal,3436134.0,...,1,Polygon,29,BARRACAS,4,13018.210271,7.961000e+06,BARRACAS,32,32
2,38029,201502,a5f052a6dea369a73d2f85326d88944659d44d86,11/27/2015,sell,house,Barracas,Argentina,Capital Federal,3436134.0,...,1,Polygon,29,BARRACAS,4,13018.210271,7.961000e+06,BARRACAS,32,32
3,51933,201502,03bb5c5811d5db8617aaf9ca8b4ce93acfa2b1eb,11/28/2015,sell,store,Barracas,Argentina,Capital Federal,3436134.0,...,1,Polygon,29,BARRACAS,4,13018.210271,7.961000e+06,BARRACAS,32,32
4,109853,201502,5e5c722f19dc028609bcdcae50bc478b3d3c9a41,11/10/2015,sell,apartment,Barracas,Argentina,Capital Federal,3436134.0,...,1,Polygon,29,BARRACAS,4,13018.210271,7.961000e+06,BARRACAS,32,32


### Export the merged data to a CSV that will be used for other merges

In [38]:
temp_df = pd.DataFrame(final_gdf)
temp_df.head(5)


,Unnamed: 0,dataset_date,id_left,created_on,operation,property_type,place_name,country_name,state_name,geonames_id,...,Uninhabited Quantile,type,index_right,barrio,comuna,perimetro,area,Barrio,id,b_id
0,1,201501,155a59477c557a77de931cb2ef91d6a9b23019d7,8/8/2014,sell,apartment,Barracas,Argentina,Capital Federal,3436134.0,...,1,Polygon,29,BARRACAS,4,13018.210271,7.961000e+06,BARRACAS,32,32
1,13547,201501,cf5823f73347ea539a10738c219624a1b9a85737,8/8/2014,sell,apartment,Barracas,Argentina,Capital Federal,3436134.0,...,1,Polygon,29,BARRACAS,4,13018.210271,7.961000e+06,BARRACAS,32,32
2,38029,201502,a5f052a6dea369a73d2f85326d88944659d44d86,11/27/2015,sell,house,Barracas,Argentina,Capital Federal,3436134.0,...,1,Polygon,29,BARRACAS,4,13018.210271,7.961000e+06,BARRACAS,32,32
3,51933,201502,03bb5c5811d5db8617aaf9ca8b4ce93acfa2b1eb,11/28/2015,sell,store,Barracas,Argentina,Capital Federal,3436134.0,...,1,Polygon,29,BARRACAS,4,13018.210271,7.961000e+06,BARRACAS,32,32
4,109853,201502,5e5c722f19dc028609bcdcae50bc478b3d3c9a41,11/10/2015,sell,apartment,Barracas,Argentina,Capital Federal,3436134.0,...,1,Polygon,29,BARRACAS,4,13018.210271,7.961000e+06,BARRACAS,32,32


### Export the merged file to CSV

In [39]:
temp_df.to_csv('properati_data/sell/sell_census_barrios.csv')

### Compute basic statistics of the data per barrio

Basic statistics: 
- Minimum price per barrio
- Maximum price per barrio
- Average price per barrio
- Standard deviation per barrio

In [26]:
filtered_gdf = final_gdf[['dataset_date', 'created_on', 'operation', 'property_type', 'place_name',\
                          'state_name', 'price', 'currency', 'price_aprox_local_currency', 'price_aprox_usd',\
                         'Commune', 'barrio','b_id', 'comuna', 'perimetro','area']]

filter_df = pd.DataFrame(filtered_gdf)

# Eliminate the properties with no price, in order not to skew the averages
filter_df = filter_df[filter_df['price_aprox_local_currency'] > 0]

In [27]:
barrios_filter_df = filter_df[['barrio','comuna','b_id', 'perimetro','area','price_aprox_local_currency', 'price_aprox_usd','created_on']]
grouped = barrios_filter_df.groupby(['b_id','barrio'], as_index=False).agg({"price_aprox_local_currency": ["min", "max", "mean", "std"], "price_aprox_usd": ["min", "max", "mean", "std"],\
                                                "created_on": "count"})

#Relabel the columns with the statistics
grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]

# Export the dataframe to a CSV
grouped.to_csv('properati_data/sell/properati_sell_grouped.csv')

In [28]:
grouped.head(5)

b_id         barrio price_aprox_local_currency                             \
                                             min          max          mean   
0    0        PALERMO                   85497.90  270456790.5  6.432754e+06   
1    1       BELGRANO                   92550.18  107992500.0  6.114209e+06   
2    2          NUÑEZ                   77468.62  206644442.4  4.963615e+06   
3    3  VILLA ORTUZAR                  798425.00  114435750.0  4.264157e+06   
4    4      BALVANERA                  101373.93   86372500.0  2.711736e+06   

                price_aprox_usd                                            \
            std             min         max           mean            std   
0  8.941423e+06         5354.16  15692300.0  377037.780885  519728.980056   
1  7.887623e+06         5256.89   5500000.0  357303.318225  456902.329404   
2  7.445223e+06         4851.34  11111111.0  290027.683553  422641.023608   
3  7.615148e+06        46817.00   6500000.0  247656.681005  438215.984322   
4  3.312287e+06         5162.92   5000000.0  159420.883301  192227.002270   

  created_on  
       count  
0      14124  
1       6642  
2       2747  
3        975  
4       5737

In [44]:
#export to JSON -- in order for us to be able to import into SQLite for the web application
grouped.to_json("properati_data/sell/properati_sell.json", orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)



In [29]:
filter_df['created_on'] = pd.to_datetime(filter_df['created_on'])
filter_df['month'] = filter_df.created_on.apply(lambda x: x.month)
filter_df['year'] = filter_df.created_on.apply(lambda x: x.year)
filter_df = filter_df[filter_df['price_aprox_local_currency'] > 0]

barrios_filter_df = filter_df[['barrio','b_id','dataset_date','comuna','perimetro','area','price_aprox_local_currency', 'price_aprox_usd',\
                             'created_on', 'month', 'year']]


In [30]:
barrios_filter_df.head(5)

,barrio,b_id,dataset_date,comuna,perimetro,area,price_aprox_local_currency,price_aprox_usd,created_on,month,year
0,BARRACAS,32,201501,4,13018.210271,7.961000e+06,3109410.00,180000.00,2014-08-08,8,2014
1,BARRACAS,32,201501,4,13018.210271,7.961000e+06,2591175.00,150000.00,2014-08-08,8,2014
2,BARRACAS,32,201502,4,13018.210271,7.961000e+06,1627587.50,95000.00,2015-11-27,11,2015
3,BARRACAS,32,201502,4,13018.210271,7.961000e+06,85275.81,4977.43,2015-11-28,11,2015
4,BARRACAS,32,201502,4,13018.210271,7.961000e+06,2826862.50,165000.00,2015-11-10,11,2015


In [31]:
grouped_month = barrios_filter_df.groupby(['barrio', 'b_id','month', 'year'], as_index=False).agg({"price_aprox_local_currency": ["min", "max", "mean", "std"], "price_aprox_usd": ["min", "max", "mean", "std"],\
                                                "created_on": "count"}).fillna(0) #for the std deviation values that resulted in NaN 
grouped_month.columns = ["_".join(x) for x in grouped_month.columns.ravel()]

In [32]:
grouped_month.head(5)

,barrio_,b_id_,month_,year_,price_aprox_local_currency_min,price_aprox_local_currency_max,price_aprox_local_currency_mean,price_aprox_local_currency_std,price_aprox_usd_min,price_aprox_usd_max,price_aprox_usd_mean,price_aprox_usd_std,created_on_count
0,AGRONOMIA,25,1,2015,1347411.00,1347411.0,1.347411e+06,0.000000e+00,78000.0,78000.0,78000.000000,0.000000,1
1,AGRONOMIA,25,1,2016,1037952.50,1437165.0,1.237559e+06,2.822859e+05,65000.0,90000.0,77500.000000,17677.669530,2
2,AGRONOMIA,25,1,2017,1916220.00,4774581.5,2.573210e+06,1.032880e+06,120000.0,299000.0,161142.857143,64682.374136,7
3,AGRONOMIA,25,1,2018,1366497.82,11388300.0,4.839287e+06,3.233757e+06,69595.0,580000.0,246462.291667,164693.502538,24
4,AGRONOMIA,25,2,2015,1209215.00,1209215.0,1.209215e+06,0.000000e+00,70000.0,70000.0,70000.000000,0.000000,1


In [42]:
#Export results to CSV and JSON
grouped_month.to_csv('properati_data/properati_sell_grouped_month.csv')
grouped_month.to_json("properati_data/properati_sell_month.json", orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)

